In [ ]:
from typing import Any, Generator, Protocol, List, Tuple
import pandas as pd
# import nltk
# nltk.download('punkt_tab')
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score
from minicheck.minicheck import MiniCheck
import json
import os
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../eval')
from utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/miaoran/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
minicheck_models = ['roberta-large', 'deberta-v3-large', 'flan-t5-large']#, 'Bespoke-MiniCheck-7B']

In [ ]:
def MinicheckEval(model_name, sources, summaries):
    model = MiniCheck(model_name=model_name, cache_dir='./ckpts')
    pred_label, raw_prob, _, _ = model.score(docs=sources, claims=summaries)
    print(pred_label)
    return pred_label


## Sample-level Prediction

In [ ]:
filename = f"../assign/examples_to_annotate.csv"
batch_size = 15
for idx, minichecker in enumerate(minicheck_models):
    print(f"Run model {str(idx)}: {minichecker} ......")
    df = pd.read_csv(filename, encoding='utf-8').fillna('')
    col_name = f'minicheck-{minichecker}'
    scores = []
    n = len(df['source'].tolist())
    for i in range(0,n,batch_size):
        pred_label = MinicheckEval(minichecker, df['source'].tolist()[i:i+batch_size], df['summary'].tolist()[i:i+batch_size])                                                                                                        batch_size])
        scores.extend(pred_label)

    if col_name in df:
        df[col_name] = scores
    else:
        df.insert(len(df.columns), col_name, scores)
    outputfile = filename
    df.to_csv(outputfile, mode='w', index=False, header=True)


## Sent-level Prediction

In [ ]:
sent_level_labels = {}
result_files, skip_sample_ids, selected_annotators, num_annotators = process_result_files()
for file_path in result_files:
    _, _, _, batch_sent_level_labels = read_annotation(file_path, skip_sample_ids=skip_sample_ids)
    # print(batch_sent_level_labels)
    sent_level_labels.update(batch_sent_level_labels)

In [ ]:
fname = '../eval/sent_level_results/detectors_sent_level_preds.json'
sources = []
df = pd.read_csv('../assign/examples_to_annotate.csv')
for index, row in df.iterrows():
    sources.append(row['source'])

for idx, minichecker in enumerate(minicheck_models):
    print(f"Run model {idx} - {minichecker}")
    existing_meta_ids = []
    data = {}
    if os.path.exists(fname):
        with open(fname) as r:
            data = json.load(r)
            for meta_id in data:
                # print(list(data[meta_id].values())[0])
                if f"minicheck-{minichecker}" in list(data[meta_id].values())[0]:
                    existing_meta_ids.append(meta_id)
                
    for meta_id in sent_level_labels:
        meta_id = str(meta_id)
        if meta_id in existing_meta_ids:
            continue
        if meta_id in data:
            item = data[meta_id]
            print(item)
        else:
            item = {}
        for sent, sent_labels in sent_level_labels[int(meta_id)].items():
            if sent not in item:
                item[sent] = {'labels': sent_labels}
            
            item[sent][f"minicheck-{minichecker}"] = MinicheckEval(minichecker, sources[int(meta_id)], sent)[0]
            
        print(item)
        if os.path.exists(fname):
            with open(fname, 'r') as f:
                json_data = json.load(f)
                json_data[meta_id] = item
        else:
            json_data = {meta_id:item}
        with open(fname, 'w') as f:
            f.write(json.dumps(json_data, indent=2))

Run model 0 - roberta-large
{"Here's a concise summary of the passage, covering the core pieces of information:\n\n": {'labels': ['Consistent'], 'alignscore-base': 0.0026, 'alignscore-large': 0.04955, 'HHEMv1': 0.36422, 'HHEM-2.1': 0.70906, 'HHEM-2.1-English': 0.74753, 'HHEM-2.1-Open': 0.79617}, 'The passage contains two unrelated pieces of information:\n\n': {'labels': ['Consistent'], 'alignscore-base': 0.05622, 'alignscore-large': 0.86291, 'HHEMv1': 0.08295, 'HHEM-2.1': 0.71762, 'HHEM-2.1-English': 0.65195, 'HHEM-2.1-Open': 0.71545}, '1. Anne Rice: \n- Born in New Orleans\n- Spent much of her early life in New Orleans\n- Later moved to Texas and then to San Francisco\n\n': {'labels': ['Consistent'], 'alignscore-base': 0.46265, 'alignscore-large': 0.50817, 'HHEMv1': 0.96417, 'HHEM-2.1': 0.98892, 'HHEM-2.1-English': 0.99657, 'HHEM-2.1-Open': 0.97752}, '2. Route 495:\n- A 3.45-mile freeway in Hudson County, New Jersey, USA\n- Connects the New Jersey Turnpike (I-95) at exits 16E and 17 i

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  3.60it/s]


[0]


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 46.32it/s]


[0]


...

[1]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  9.58it/s]


[1]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  9.33it/s]


[1]
{' The passage discusses a study by the Institute for Health Metrics and Evaluation at the University of Washington, published in the American Journal of Public Health, which found an increase in heavy drinking and binge-drinking among Americans between 2005 and 2012. ': {'labels': ['Consistent'], 'alignscore-base': 0.76417, 'alignscore-large': 0.76526, 'HHEMv1': 0.30142, 'HHEM-2.1': 0.99316, 'HHEM-2.1-English': 0.99904, 'HHEM-2.1-Open': 0.98043, 'minicheck-roberta-large': 1, 'minicheck-deberta-v3-large': 1, 'minicheck-flan-t5-large': 1}, 'Heavy drinking is defined as exceeding an average of one drink per day for women and two drinks per day for men, while binge drinking is defined as four or more drinks for women and five or more drinks for men on a single occasion. ': {'labels': ['Unwanted.Instrinsic', 'Unwanted'], 'alignscore-base': 0.3813, 'alignscore-large': 0.77914, 'HHEMv1': 0.30089, 'HHEM-2.1': 0.99585, 'HHEM-2.1-English': 0.99869, 'HHEM-2.1-Open': 0.97516, 'minicheck-rober